[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mravanba/comp551-notebooks/blob/master/GradientDescent.ipynb)

# Implementing Minibatch Gradient Descent for Neural Networks

Source: https://wiseodd.github.io/techblog/2016/06/21/nn-sgd/

Learn how to create a simple 3-layers Neural Networks, and implement Minibatch Gradient Descent using Numpy.

## 1. Define the Networks

In [46]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import numpy as np
import random

In [47]:
X, y = make_moons(n_samples=5000, random_state=42, noise=0.1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, shuffle=True)

In [48]:
# define the NN model (Three layers networks. Single hidden layer)

n_feature = 2
n_class = 2
n_iter = 10

def make_network(n_hidden=100):
    model = dict(
        W1=np.random.randn(n_feature, n_hidden),
        W2=np.random.randn(n_hidden, n_class)
    )
    return model

In [49]:
## Define two operations: forward and back propagations
# 1. Define forward propagation

def softmax(x):
    return np.exp(x) / np.exp(x).sum()


def forward(x, model):
    # Input to hidden
    h = x@model['W1']
    # ReLU non-linearity
    h[h<0] = 0  # this is effectively f(x)=max(0, x)

    # Hidden to output
    prob = softmax(h@model['W2'])

    return h, prob

Once we reach the output layer, we need to make the output to be a probability distribution, Bernoulli to be exact. Therefore, we squash the output with Softmax function to get our label distribution.

In [61]:
def backward(model, xs, hs, errs):
    """ xs, hs, errs contain all info (input, hidden state, error) of all data in the minibatch"""

    # errs in the gradients of output layer for the minibatch
    dW2 = hs.T@errs

    # Get gradient of hidden layer
    dh = errs@model['W2'].T
    dh[hs<=0] = 0

    dW1 = xs.T@dh

    return dict(W1=dW1, W2=dW2)

## 2. Gradient Descent

Gradient Descent is the most popular optimization algorithm in machine learning.



*   Gradient descent is a first order method => we only need to compute the gradient of the objective function to use it. <=> second order method, where we need to derive the partial second order derivative matrix (Hessian)







There are three variants of Gradient Descent:


1.   **Batch** gradient descent: Uses full training data at each iteration, which could be very expensive if the dataset is large and if the networks have a lot of parameters.
2.   **Stochastic gradient descent**: Uses only single data point to propagate the error, which would make the convergence slow, as the variance is big.
3.   **Minibatch gradient descent**: This is a combination of the best of both batch and stochastic gradient descents. We neither use full dataset nor a single data point. For example, we use 50, 100, or 200 random *subset* of the dataset each time we train the networks(model). This way, we lower the computational cost, and yet, we still get lower variance than by using the Stochastic gradient descent.

## Implementing the Minibatch Gradient Descent
Note that Minibatch Gradient Descent is an *algorithm*.

In [51]:
def sgd(model, X_train, y_train, minibatch_size):
    for iter in range(n_iter):
        print("Iteration {}".format(iter))
    
    # Randomize datapoint
    #X_train, y_train = shuffle(X_train, y_train)

    # Get chunks/minibatches
    for i in range(0, X_train.shape[0], minibatch_size):
        # Get pair of (X, y) of the current minibatch/chunk
        X_train_mini = X_train[i:i+minibatch_size]
        y_train_mini = y_train[i:i+minibatch_size]

        # Feed a minibatch to the network/model
        # The graidents of that minibatch will be propagated back to updated the parameter/weight of the network
        model = sdg_step(model, X_train_mini, y_train_mini)
    
    return model

In [52]:
def sdg_step(model, X_train, y_train):
    
    # Get the gradients of each layer of the networks for the current minibatch
    grad = get_minibatch_grad(model, X_train, y_train)
    model = model.copy()

    # Update every parameters in our networks (W1 and W2) using their gradients
    for layer in grad:
        l_rate = 1e-4
        model[layer] += l_rate*grad[layer]  # Update weights of each layer by adding the gradient of particular weight matrix to the existing weight matrix
    
    return model

When thinking visually, **gradient of a function** is the *direction of our steps*.

**Learning rat**e would be how far we take the step.

In [59]:
def get_minibatch_grad(model, X_train, y_train):
    xs, hs, errs = [], [], []

    for x, cls_idx in zip(X_train, y_train):
        h, y_pred = forward(x, model)

        # Create probability distribution of true label
        y_true = np.zeros(n_class)
        y_true[int(cls_idx)] = 1

        # Compute the gradient of output layer
        err = y_true - y_pred

        # Accumulate the info of minibatch
        # x: input
        # h: hidden state
        # err: gradient of output layer
        xs.append(x)
        hs.append(h)
        errs.append(err)

    # Back propagate using the info we got from the current minibatch
    return backward(model, np.array(xs), np.array(hs), np.array(errs))

What's happening above: 


1.   Iterate through every data point in a minibatch
2.   Feed it to the network and compare the output with the true label from the training data.
3.  The error is neatly defined by the difference of the probability of true label with the probability of our prediction. This is so, bc we're implicitly using the Softmax output with Cross Entropy cost function.


Because this is a Minibatch GD alg, we then accumulate all the info of the current minibatch. We use all those info of the current minibatch to do the back propagation, which will yield gradients of the networks' parameters (W1 and W2)


## Testing
Let's test our model and algorithm.
We'll repeat the train-test procedure for 100 times then report the average of the accuracy.

In [63]:
minibatch_size = 50
n_experiment = 100

# Create a placeholder to accumulate prediction accuracy
accs = np.zeros(n_experiment)

for k in range(n_experiment):
    # Reset model
    model = make_network()

    # Train the model
    model = sgd(model, X_train, y_train, minibatch_size)

    y_pred = np.zeros_like(y_test)

    for i, x in enumerate(X_test):
        # Predict the distribution of label
        _, prob = forward(x, model)
        # Get label by picking the most probable one
        y = np.argmax(prob)
        y_pred[i] = y

    # Compare the predictions with the true labels and take the percentage
    accs[k] = (y_pred == y_test).sum() / y_test.size

print("Mean accuracy: {}, std: {}".format(accs.mean(), accs.std()))

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 0
Iteration 1
Iteration 2
Iter